## This tutorial is how to map coordinates between source and target slides
Author: Minji Kim  
Creation date: 10/24/2022  
If you have any questions, please email me: minjikim.cs@gmail.com  

## 0. Import packages

In [1]:
import os
from pathlib import Path
import pandas as pd
from typing import Dict

## 1. Read deformField file & Prepare transformation dictionary
After pixel by pixel registration on thumbnail images, we can get deformField.csv file.  For this example, the H&E and FoxP3 slides are used with Image_id (1664369_MR16-1693).   Source means H&E slide and target means IHC FoxP3 slide (target images are moving for registartion). Transformation dictionary contains corresponding keys (source thumbnail coordinates) and values (target thumbnail coordinates)

In [2]:
def prepare_transform_dict(deformField_file_path:str) -> Dict:
    # prepare transformation dictionary from deformField file    
    
    # read deformField file
    deformed_df = pd.read_csv(deformField_file_path)
    
    # drop null values
    deformed_df.dropna(inplace=True)
    
    # convert data type into int
    deformed_df = deformed_df.astype({"source_x":'int', "source_y":'int', "target_x":'int', "target_y":'int'}) 
    
    # prepare source coordinates and target coordinates    
    source_coords = list(zip(deformed_df['source_x'], deformed_df['source_y'])) 
    target_coords = list(zip(deformed_df['target_x'], deformed_df['target_y'])) 

    # prepare transformation dictionary keys: source coordinates, values: target coordinates
    transform_dict = dict(zip(source_coords, target_coords))    
    return transform_dict

In [3]:
# create transformation dictionary
transform_dict = prepare_transform_dict('Van_Abel_all_thumbnail_images/1664369/registration_Tumor_HE_FoxP3/deformField.csv')

## 2. Read H&E and IHC information file & Mapping using transformation dictionary
(Note) All WSIs are tiled by 512x512 px and Thumbnail tiles are 1x1 px.

In [4]:
# define source and target tiled information file paths
source_file_path = 'Van_Abel_all_thumbnail_images/1664369/1664369_MR16-1693 J3_Tumor_HE___reference_wsi-tilesize_x-512-y-512_mask-tilesize_x-1-y-1_img-level_0.tsv'
target_file_path = 'Van_Abel_all_thumbnail_images/1664369/1664369_MR16-1693 J3_Tumor_FoxP3___reference_wsi-tilesize_x-512-y-512_mask-tilesize_x-1-y-1_img-level_0.tsv'

# read source and target information file
source_df = pd.read_csv(source_file_path, sep='\t')
target_df = pd.read_csv(target_file_path, sep='\t')

# drop non-tile rows
source_df = source_df[source_df.filename.notnull()]
target_df = target_df[target_df.filename.notnull()]

# reset index
source_df.reset_index(inplace=True, drop=True)
target_df.reset_index(inplace=True, drop=True)

# add prefix for each dataframe
source_df = source_df.add_prefix('source_')
target_df = target_df.add_prefix('target_')

In [5]:
source_df

,source_image_id,source_tile_id,source_index_x,source_index_y,source_wsi_x,source_wsi_y,source_mask_x,source_mask_y,source_filename,source_tissue_ratio
0,1664369_MR16-1693 J3_Tumor_HE,2171,14,99,7256,50875,14,99,1664369_MR16-1693 J3_Tumor_HE__x7256_y50875.png,0.218750
1,1664369_MR16-1693 J3_Tumor_HE,2172,14,100,7256,51387,14,100,1664369_MR16-1693 J3_Tumor_HE__x7256_y51387.png,0.375000
2,1664369_MR16-1693 J3_Tumor_HE,2173,14,101,7256,51899,14,101,1664369_MR16-1693 J3_Tumor_HE__x7256_y51899.png,0.437500
3,1664369_MR16-1693 J3_Tumor_HE,2174,14,102,7256,52411,14,102,1664369_MR16-1693 J3_Tumor_HE__x7256_y52411.png,0.703125
4,1664369_MR16-1693 J3_Tumor_HE,2175,14,103,7256,52923,14,103,1664369_MR16-1693 J3_Tumor_HE__x7256_y52923.png,0.703125
...,...,...,...,...,...,...,...,...,...,...
17107,1664369_MR16-1693 J3_Tumor_HE,36635,247,79,126552,40635,247,79,1664369_MR16-1693 J3_Tumor_HE__x126552_y40635.png,0.812500
17108,1664369_MR16-1693 J3_Tumor_HE,36636,247,80,126552,41147,247,80,1664369_MR16-1693 J3_Tumor_HE__x126552_y41147.png,0.125000
17109,1664369_MR16-1693 J3_Tumor_HE,36782,248,78,127064,40123,248,78,1664369_MR16-1693 J3_Tumor_HE__x127064_y40123.png,0.093750
17110,1664369_MR16-1693 J3_Tumor_HE,36783,248,79,127064,40635,248,79,1664369_MR16-1693 J3_Tumor_HE__x127064_y40635.png,0.234375


In [6]:
target_df

,target_image_id,target_tile_id,target_index_x,target_index_y,target_wsi_x,target_wsi_y,target_mask_x,target_mask_y,target_filename,target_tissue_ratio
0,1664369_MR16-1693 J3_Tumor_FoxP3,2255,14,99,7206,50733,14,99,1664369_MR16-1693 J3_Tumor_FoxP3__x7206_y50733...,0.078125
1,1664369_MR16-1693 J3_Tumor_FoxP3,2256,14,100,7206,51245,14,100,1664369_MR16-1693 J3_Tumor_FoxP3__x7206_y51245...,0.093750
2,1664369_MR16-1693 J3_Tumor_FoxP3,2257,14,101,7206,51757,14,101,1664369_MR16-1693 J3_Tumor_FoxP3__x7206_y51757...,0.078125
3,1664369_MR16-1693 J3_Tumor_FoxP3,2258,14,102,7206,52269,14,102,1664369_MR16-1693 J3_Tumor_FoxP3__x7206_y52269...,0.046875
4,1664369_MR16-1693 J3_Tumor_FoxP3,2407,15,97,7718,49709,15,97,1664369_MR16-1693 J3_Tumor_FoxP3__x7718_y49709...,0.015625
...,...,...,...,...,...,...,...,...,...,...
8473,1664369_MR16-1693 J3_Tumor_FoxP3,48043,311,149,159270,76333,311,149,1664369_MR16-1693 J3_Tumor_FoxP3__x159270_y763...,1.000000
8474,1664369_MR16-1693 J3_Tumor_FoxP3,48044,311,150,159270,76845,311,150,1664369_MR16-1693 J3_Tumor_FoxP3__x159270_y768...,1.000000
8475,1664369_MR16-1693 J3_Tumor_FoxP3,48045,311,151,159270,77357,311,151,1664369_MR16-1693 J3_Tumor_FoxP3__x159270_y773...,1.000000
8476,1664369_MR16-1693 J3_Tumor_FoxP3,48046,311,152,159270,77869,311,152,1664369_MR16-1693 J3_Tumor_FoxP3__x159270_y778...,1.000000


In [7]:
def map_coords(source_df:pd.DataFrame, target_df:pd.DataFrame, transform_dict:Dict) -> pd.DataFrame:
    
    # before mapping, modify target thumbnail coordinates by substracting offset values
    source_df['source_mask_coords']  = list(map(tuple, zip(source_df["source_mask_x"], source_df["source_mask_y"])))

    # mapping target thumbnail coordinate and source coordincates
    source_df["target_mask_coords"] = source_df['source_mask_coords'].apply(lambda x: transform_dict.get(x))

    # before mapping, modify target thumbnail coordinates by substracting offset values
    target_df['target_mask_coords']  = list(map(tuple, zip(target_df["target_mask_x"], target_df["target_mask_y"])))
    
    merge_df = source_df.merge(target_df, on = 'target_mask_coords', how='left')
    merge_df.drop(columns = ['source_mask_coords', 'target_mask_coords'], inplace=True)
    
    return merge_df

In [8]:
merge_df = map_coords(source_df, target_df, transform_dict)

In [9]:
merge_df

,source_image_id,source_tile_id,source_index_x,source_index_y,source_wsi_x,source_wsi_y,source_mask_x,source_mask_y,source_filename,source_tissue_ratio,target_image_id,target_tile_id,target_index_x,target_index_y,target_wsi_x,target_wsi_y,target_mask_x,target_mask_y,target_filename,target_tissue_ratio
0,1664369_MR16-1693 J3_Tumor_HE,2171,14,99,7256,50875,14,99,1664369_MR16-1693 J3_Tumor_HE__x7256_y50875.png,0.218750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1664369_MR16-1693 J3_Tumor_HE,2172,14,100,7256,51387,14,100,1664369_MR16-1693 J3_Tumor_HE__x7256_y51387.png,0.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1664369_MR16-1693 J3_Tumor_HE,2173,14,101,7256,51899,14,101,1664369_MR16-1693 J3_Tumor_HE__x7256_y51899.png,0.437500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1664369_MR16-1693 J3_Tumor_HE,2174,14,102,7256,52411,14,102,1664369_MR16-1693 J3_Tumor_HE__x7256_y52411.png,0.703125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1664369_MR16-1693 J3_Tumor_HE,2175,14,103,7256,52923,14,103,1664369_MR16-1693 J3_Tumor_HE__x7256_y52923.png,0.703125,1664369_MR16-1693 J3_Tumor_FoxP3,2255.0,14.0,99.0,7206.0,50733.0,14.0,99.0,1664369_MR16-1693 J3_Tumor_FoxP3__x7206_y50733...,0.078125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17107,1664369_MR16-1693 J3_Tumor_HE,36635,247,79,126552,40635,247,79,1664369_MR16-1693 J3_Tumor_HE__x126552_y40635.png,0.812500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17108,1664369_MR16-1693 J3_Tumor_HE,36636,247,80,126552,41147,247,80,1664369_MR16-1693 J3_Tumor_HE__x126552_y41147.png,0.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17109,1664369_MR16-1693 J3_Tumor_HE,36782,248,78,127064,40123,248,78,1664369_MR16-1693 J3_Tumor_HE__x127064_y40123.png,0.093750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17110,1664369_MR16-1693 J3_Tumor_HE,36783,248,79,127064,40635,248,79,1664369_MR16-1693 J3_Tumor_HE__x127064_y40635.png,0.234375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
